<a href="https://colab.research.google.com/github/HedersonSantos/nlp_nao_supervisionado/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install torch transformers
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [69]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

In [18]:
textos = ['''Correios Log+
O serviço Correios Log+ suporta operações logísticas completas, com armazenagem, atendimento de pedidos e integração aos demais serviços de entrega e logística reversa dos Correios.
Também chamado de fulfillment, é destinado às empresas que atuam na venda de produtos para os mercados nacional e internacional, que poderão contar com um único operador logístico capaz de fornecer todo suporte necessário e reconhecido pela sua liderança no mercado de entregas.
Empresas que atuam (ou desejam atuar) no comércio eletrônico por meio de site próprio ou marketplaces podem contratar o serviço Correios Log+.
É por meio da formalização de parceria com os Correios, através da celebração do contrato comercial ou inclusão do serviço no contrato já existente, que os clientes poderão iniciar suas operações no Correios Log.
A contratação dos Correios poderá ser feita na internet, por meio do Correios Fácil, ou diretamente com os Assistentes Comerciais.
Os Correios assumem as etapas mais onerosas do processo logístico no comércio eletrônico, que é a armazenagem e o manuseio, bem como a logística internacional, quando for o caso. Isso permite que as empresas ganhem celeridade no processo operacional ao transferir estas responsabilidades para um único operador logístico.
Após incluir o serviço no contrato, analise quais são as localidades que melhor atendem suas necessidades.
Para que o cliente possa armazenar os produtos nos Centros dos Correios, a legislação brasileira requer que o cliente crie uma filial (fiscal) em cada estado onde desejar realizar operação.
Cidades onde o serviço será prestado:
Brasília DF:
Cajamar SP:
Rio de Janeiro RJ:
Contagem MG:
Curitiba PR:
Florianópolis SC:
Fortaleza CE:
Manaus AM:
Porto Alegre RS:
Recife PE:
Salvador BA:
Goiânia - GO;
Ribeirão Preto - SP
Para a execução do serviço Correios Log+, é preciso utilizar um sistema de ERP integrado aos Correios por meio das APIs do Correios Log+.
Com a integração dos sistemas tecnológicos, os Correios recebem os pedidos, separam os produtos disponíveis em estoque, embalam e preparam para o envio, com impressão de etiquetas de postagens e documentos fiscais necessários conforme legislação vigente. ''',
'''Emissão CPF
O que é?
É o serviço que permite ao cidadão solicitar CPF, alterar dados cadastrais e regularizar a situação do CPF suspenso nas agências dos Correios do país.
Quem pode utilizar este serviço?
Pessoas físicas e órgãos públicos com contrato a faturar com os Correios para atendimento aos cidadãos que não têm condições de pagar pelo serviço.
Como funciona?
Para brasileiros com idade dos 18 aos 69 anos: título de eleitor, alistamento eleitoral, protocolo de inscrição ou certidão da Justiça Eleitoral atestando a não obrigatoriedade do alistamento eleitoral), solicitar o serviço e pagar a taxa.
Você sai da agência com o novo CPF e com as solicitações referentes ao serviço atendidas no mesmo instante.
E para o menor de 16 anos, tutelado, curatelado ou outra pessoa sujeita à guarda judicial?
A solicitação do CPF deverá ser feita pelo representante legal, que pode ser pai ou mãe, tutores, curadores ou responsáveis pela guarda judicial; Carteira de identidade ou certidão de nascimento que comprove a naturalidade, a filiação e a data de nascimento do menor, tutelado ou curatelado; e documentos que atestem a tutela, curatela ou responsabilidade pela guarda de incapaz ou interdito.
E quem tiver entre 16 e 18 anos incompletos?
São assistidos legalmente, podendo assinar por seus atos ou não. Há uma Instrução Normativa da Receita Federal que estabelece que nesses casos pai ou mãe poderão assinar pela solicitação do CPF.
Se for estrangeiro?
Poderá ser aceito como documento de identificação o que é admitido no seu país de origem. Também poderão ser apresentados o RNE (Registro Nacional de Estrangeiro), o passaporte, ou o protocolo do RNE acompanhado da tela de consulta impressa do SINCRE - Sistema Nacional de Estrangeiros, constando os dados cadastrais. Não é obrigatória a comprovação de filiação de estrangeiros.
Veja o que fazer se for alterar ou regularizar os dados do CPF:
É possível solicitar gratuitamente a correção dos dados do CPF em até 60 dias. Vá até uma agência dos Correios dentro desse prazo e formalize a alteração;
• Se você casou e quer alterar seu nome, deve apresentar a certidão de casamento, mas se for fazer a comprovação da data de nascimento basta apresentar a carteira de identidade;
• Para solicitar alteração de endereço não é preciso apresentar comprovante de residência;
• Os brasileiros ou estrangeiros que moram em outro país ou aqueles residentes no Brasil que estejam em viagem ao exterior podem solicitar alteração de dados cadastrais;
• Quem estiver com a situação cadastral "pendente de regularização" ou "suspensa" precisa fazer a regularização mesmo que não tenha sido obrigado a entregar a Declaração do Imposto de Renda nos últimos cinco anos.
Quanto custa?
R$7,00 (sete reais)''',
'''Tipo de Atendimento:
Conclusivo: é o atendimento realizado exclusivamente pelos Correios, sem a necessidade do cliente comparecer à unidade local da Secretaria Receita Federal do Brasil.
Não conclusivo: é o atendimento realizado inicialmente nos Correios e concluído pela Secretaria Receita Federal do Brasil. Os clientes enquadrados em uma das situações específicas, relacionadas a seguir, receberão Atendimento Não Conclusivo:
a) Pessoas com idade igual ou superior a 25 (vinte cinco) anos;
b) Homônimo Perfeito;
c) Nome mãe = "mãe desconhecida" ou em branco;
d) Data nascimento não oficial: pessoa com data de nascimento inválida registrada na documentação oficial;
e) Nome contribuinte inválido: pessoa registrada com apenas o prenome, sem sobrenome, com nome com mais de quinze partes, abreviado, com caracteres não válidos;
f) Nome mãe inválido: pessoa registrada com nome da mãe com apenas o prenome, sem sobrenome, com nome com mais de quinze partes, abreviado, com caracteres não válidos;
g) Título de Eleitor já utilizado por outro CPF;
h) CPF não encontrado na base ou (para inclusão) não pertence à faixa MIA (Modelo de inscrição e Atualização);
i) Título de Eleitor não informado, mas informado motivo de ausência;
j) Município de naturalidade = 9707 e sem indicativo de estrangeiro na base CPF;
k) CPF em situação cadastral cancelada ou nula;
l) CPF de responsável legal inválido, não encontrado na base CPF ou na situação cancelada ou nula;
m) Título de Eleitor diverge da base;
n) Nome e/ou nascimento diverge com a base RFB;
o) Nome da mãe não pode ser alterado (mãe desconhecida, em branco, divergente da base TSE) e/ou Sexo não pode ser alterado.
Impeditivo: é aquele que apresenta alguma inconsistência no atendimento, não permitindo a finalização do atendimento. Os clientes enquadrados em uma das situações específicas, relacionadas a seguir, receberão Atendimento Impeditivo:
a) Solicita o serviço de correção, mas a última operação no CPF não é da conveniada solicitante. (Essa situação irá ocorrer se uma pessoa solicitar um serviço (Inscrição ou Alteração) nos agentes conveniados Banco do Brasil - BB ou Caixa Econômica Federal - CEF e depois pedir a correção nos Correios;
b) Solicita alteração, mas os dados informados estão iguais aos da base CPF;
c) Solicita correção cadastral, mas os dados informados estão iguais aos da base
CPF; d) Solicita regularização, mas o CPF já está regular;
e) Solicita regularização, mas CPF consta como omisso de DIRPF. ''']

In [8]:
for t in textos:
  print(len(t.split(' ')))

324
425
370


In [70]:
class textoDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Definimos labels como input_ids, isso é necessário para Masked Language Modeling
        item['labels'] = item['input_ids'].clone()
        return item

In [72]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [73]:
# Tokenização dos textos
# Dataset e DataLoader
dataset = textoDataset(textos, tokenizer)
#train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [74]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [60]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    overwrite_output_dir=True,
    output_dir='./gpt2_saida'


)#prediction_loss_only=True
#eval_strategy="epoch",

In [75]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,

)
#compute_metrics=compute_loss
#eval_dataset=eval_dataset

In [76]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=4.326728185017903, metrics={'train_runtime': 3.4352, 'train_samples_per_second': 2.62, 'train_steps_per_second': 0.873, 'total_flos': 4675698432000.0, 'train_loss': 4.326728185017903, 'epoch': 3.0})

In [77]:
# Salve o modelo ajustado
model.save_pretrained('./finetuned_gpt2')
tokenizer.save_pretrained('./finetuned_gpt2')

('./finetuned_gpt2/tokenizer_config.json',
 './finetuned_gpt2/special_tokens_map.json',
 './finetuned_gpt2/vocab.json',
 './finetuned_gpt2/merges.txt',
 './finetuned_gpt2/added_tokens.json')

In [78]:
!ls -lh finetuned_gpt2/

total 477M
-rw-r--r-- 1 root root  907 Jun 15 20:29 config.json
-rw-r--r-- 1 root root  119 Jun 15 20:29 generation_config.json
-rw-r--r-- 1 root root 446K Jun 15 20:29 merges.txt
-rw-r--r-- 1 root root 475M Jun 15 20:29 model.safetensors
-rw-r--r-- 1 root root  470 Jun 15 20:29 special_tokens_map.json
-rw-r--r-- 1 root root  525 Jun 15 20:29 tokenizer_config.json
-rw-r--r-- 1 root root 976K Jun 15 20:29 vocab.json


In [84]:
from transformers import pipeline

# Carrega o modelo ajustado para geração de texto
model_path = "./finetuned_gpt2"  # Atualize com o caminho correto
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path, truncation=True)
# Define o token de padding
tokenizer.pad_token = tokenizer.eos_token

# Cria um pipeline para geração de texto
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Exemplo de uso do modelo ajustado
prompt = "Em português, o que é o Correios Log+?"
generated_text = generator(prompt, max_length=100, num_return_sequences=1)

print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Em português, o que é o Correios Log+? —? tl! — 1 1 <e> — d <e> — y <e> u <u> u <u> (T, v) e <e> o <e> — e> — s <e> <e> o <e> — o <e> o <e> — o <e> O <e> » 2 <e> 4 <e>'}]


In [87]:
texto = "Este é um exemplo de texto em português que será tokenizado."
inputs = tokenizer(texto, return_tensors="pt")
inputs

{'input_ids': tensor([[   36,  4169, 38251, 23781, 21433,    78,   390,  2420,    78,   795,
          2493, 45284, 25792,    82,  8358,  1055,  6557, 11241,   528,  4533,
            13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}